## Поиск похожих городских маршрутов

В данном блокноте приведена реализация алгоритма поиска похожих городских маршрутов. Алгоритм базируется на поиске наибольшей общей последовательности перегонов у двух маршрутов.

Особенностью транспортной системы Санкт-Петербурга является то, что у разного вида транспорта остановки могут быть разными даже если они находятся в непосредственной близости (например, выделенные трамвайные и троллейбусные остановки или группа остановок около станции метро). Поэтому имеет смысл объединить такие остановки в кластеры. Тогда общим перегоном у двух маршрутов будем считать такой перегон, у которого начальные остановки лежат в одном кластере и конечные остановки также лежат в одном кластере.

Пример перегона, который должен быть определен как общий, можно увидеть на следующем рисунке. Здесь изображены трамваный маршрут №55 (сверху) и автобусный маршрут №126 (снизу), а общий перегон выделен красным прямоугольником. 

![similar_routes1.jpg](resources/img/similar_routes1.jpg)

![similar_routes2.jpg](resources/img/similar_routes2.jpg)

Корректность такого определения общих перегонов базируется на том факте, что человек, у которого нет предпочтений в выборе транспорта и которому нужно добраться, например, от метро Комендантский проспект до проспекта Авиаконструкторов, может выбрать как трамвай №55, так и автобус №126. Они оба являются подходщими для него и в его понимании следуют эту остановку по одному маршруту, несмотря на то, что ни начальные, ни конечные остановки у них полностью не совпадают (но они находятся рядом и принадлежат одним кластерам). 

Поэтому еще до реализации алгоритма в данные об остановках на маршрутах городского и коммерческого транспорта были также добавлены данные о кластерах, которым эти остановки принадлежат.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Загружаем подготовленные датасеты с городскими и коммерческими маршрутами 
# (с полными списками остановок в обоих направлениях и номерами кластеров, к которым относятся остановки)
r_df = pd.read_csv('resources/social_routes_info.csv', index_col=0)
all_routes_df = r_df[['route_number', 'id_transport', 'number_of_stops_in_direction1', 'stops_in_direction1', 'number_of_stops_in_direction2', 'stops_in_direction2', 'clusters_in_direction1', 'clusters_in_direction2']]

comm_df = pd.read_csv('resources/commercial_routes.csv', index_col=0)

## Алгоритм поиска похожих городских маршрутов

Маршруты будем считать тем более похожими, чем больше у них общих перегонов. Основной целью алгоритма является поиск для конкретного заданного коммерческого маршрута таких городских маршрутов, которые больше всего на него похожи (т.е. с которыми у заданного комерческого больше всего общих перегонов).

#### Описание алгоритма

###### Входные данные: 
<номер коммерческого маршрута, тип вывода результата алгоритма, настраиваемый параметр алгоритма>

Типов вывода результата два. 
 - Если передан тип "number", то вторым параметром необходимо указать интересуемое количество наиболее похожих маршрутов, которые будут возвращены алгоритмом.
 - Если передан тип "percent", то вторым параметром необходимо указать интересуемый процент похожести для выводимых маршрутов (не менее).

###### Этапы работы:

1. Перебор всех городских маршрутов в общем списке маршрутов, для каждого выполнить 2.

2. Нахождение всех непересекающихся последовательностей общих перегонов у двух маршрутов
  1. Находится максимальная общая последовательность перегонов у двух маршрутов и добавляется в массив цепочек общих перегонов. 
  2. Найденная общая цепочка перегонов «вырезается» из обоих маршрутов и производится «склейка» оставшихся частей маршрутов. 
  3. Действия 1 и 2 повторяются на новых цепочках, пока одна из них не станет длины 0 или пока у них не закончатся общие перегоны.
  
 В результате выполнения шагов 1 и 2 для каждой пары (переданный коммерческий маршрут, городской маршрут) создается список общих последовательностей перегонов (отсортированный по убыванию длин таких последовательностей). 
  
3. Выявление наиболее похожих городских маршрутов. Сортировка по количеству совпадающих перегонов.

###### Выходные данные: 
<в зависимости от переданного типа вывода результата либо указанное количество наиболее похожих маршрутов, либо маршруты, которые прошли порог по проценту похожести>

#### Детали реализации

Реализацией шага A. на втором этапе является основная функция $lcs$ и две вспомогательные функции $sequential\_slice$ и $sequence\_in\_list$. В них с помощью итераторов и функции zip осуществляется поиск наибольшей общей последовательности кластеров остановок у двух маршрутов.

Функция $create\_list\_of\_subroutes$ реализует шаги B. и С. алгоритма. В ней происходит удаление наденной общей цепочки перегонов из обоих маршрутов, склейка и повторный запуск функции $lcs$, а также проверка на то, остались ли у рассматриваемых маршрутов общие перегоны.

В функции $most\_similar$ происходит реализация 1-ого этапа (перебор всех городских маршрутов) и части 3-его этапа (сортировка по похожести). Дополнительно исследуется, какие именно направления движения у переданного коммерческого маршрута и рассматриваемого городского совпадают. Эта информация сохраняется.

Функция $find\_most\_similar\_routes$ обобщает в себе все описанные выше функции. В ней происходит обработка переданных аргументов, анализ совпадающих направлений движения и отбор необходимого числа похожих маршрутов, а также оформление итогового отчета.

In [ ]:
# Основной алгоритм поиска городских маршрутов, которые похожи на указанный коммерческий. 
# В его основе лежит поиск наибольшей общей подпоследовательности у двух маршрутов
# Для указанного коммерческого маршрута просматриваются все доступные городские и находятся все общие последовательности 
# перегонов между каждой парой таких маршрутов. Перегон считается общим, если, соответственно, 
# пары начальных и конечных остановок коммерческого и городского маршрутов находятся в одних кластерах.

# Следующие две функции являются вспомогательными к функции lcs, которая находит максимальную общую подпоследовательность 
def sequential_slice(iterable, length):
    pool = tuple(iterable)
    assert 0 < length <= len(pool)
    tails = (pool[s:] for s in range(length))
    return zip(*tails)

def sequence_in_list(sequence, lst):
    pool = tuple(sequence)
    return any((pool == s for s in sequential_slice(lst, len(pool))))

# Непосредственно, функция, которая ищет максимальную общую подпоследовательность двух переданных строк 
# с кластерами остановок на маршрутах
def lcs(str1, str2):
    a = str1.split(',')
    b = str2.split(',')
    
    if len(a) > len(b):
        a, b = b, a
    for l in reversed(range(1, len(a)+1)):
        seq = [subseq for subseq in sequential_slice(a, l) if sequence_in_list(subseq, b)]
        if seq:
            break
    return seq

# Функция, обобщающая предыдущую. Результатом ее работы является полный список всех общих перегонов
def create_list_of_subroutes(str1, str2):
    res = 1
    all_subroutes = []
    while len(str1) > 0 and len(str2) > 0 and res != 0:
        seq = lcs(str1, str2)
        # Если нет общих перегонов или остались только общие остановки, мы завершаем работу 
        # (больше нет общих перегонов)
        if len(seq) == 0 or len(seq[0]) == 1:
            res = 0
        # Иначе мы добавляем наибольший общий в итоговый список всех общих и удаляем из обоих маршрутов
        # эту последовательность
        else:
            all_subroutes.append(seq[0])
            substr = seq[0][0]
            for i in seq[0][1:]:
                substr += "," + i
            # удаление общей подпоследовательности из маршрутов
            new_str1_array = str1.split(substr)
            if len(new_str1_array) == 1:
                str1 = str1.split(substr)[0]
                str1 = str1.replace(',,',',')
            else:
                str1 = str1.split(substr)[0] + str1.split(substr)[1]
                str1 = str1.replace(',,',',')
                
            new_str2_array = str2.split(substr) 
            if len(new_str2_array) == 1:
                str2 = str2.split(substr)[0]
                str2 = str2.replace(',,',',')
            else:
                str2 = str2.split(substr)[0] + str2.split(substr)[1]
                str2 = str2.replace(',,',',')
    amount_of_similar = 0 # Переменная, в которую мы запишем количество общих перегонов (чтобы в дальнешем сортировать)           
    for i in range(len(all_subroutes)):
        amount_of_similar += len(all_subroutes[i]) - 1
    return all_subroutes, amount_of_similar  

# Функция, которая возвращает number_of_similar городских маршрутов, которые больше всего похожи на наш

def most_similar(comm_route, number_of_similar):
    comm_clusters1 = comm_df[comm_df['route_number'] == comm_route]['clusters_in_direction1'].values[0]
    comm_clusters2 = comm_df[comm_df['route_number'] == comm_route]['clusters_in_direction2'].values[0]
    
    # Создаем zip с парами номер маршрута - тип транспорта для использования в алгоритме
    all_routes_names = all_routes_df['route_number'].values
    all_routes_types = all_routes_df['id_transport'].values
    all_routes = zip(all_routes_names, all_routes_types)
    
    choosen_direction = 1
    direction_of_result = 1
    
    similarity_arr = []
    max_subroute = []
    for route_name, route_type in all_routes:
        if comm_clusters1 != '-1':
            all_routes_this_type = all_routes_df[all_routes_df['id_transport'] == route_type]
            current_clusters_1 = all_routes_this_type[all_routes_this_type['route_number'] == route_name]['clusters_in_direction1'].values[0]
            current_clusters_2 = all_routes_this_type[all_routes_this_type['route_number'] == route_name]['clusters_in_direction2'].values[0]
            subroutes1, amount1 = create_list_of_subroutes(comm_clusters1, current_clusters_1)
            subroutes2, amount2 = create_list_of_subroutes(comm_clusters1, current_clusters_2)
            if amount2 > amount1:
                subroutes_1 = subroutes2
                amount_1 = amount2
                direction_of_result = 2
            else:
                subroutes_1 = subroutes1
                amount_1 = amount1
                direction_of_result = 1
        if comm_clusters2 != '-1':
            all_routes_this_type = all_routes_df[all_routes_df['id_transport'] == route_type]
            current_clusters_1 = all_routes_this_type[all_routes_this_type['route_number'] == route_name]['clusters_in_direction1'].values[0]
            current_clusters_2 = all_routes_this_type[all_routes_this_type['route_number'] == route_name]['clusters_in_direction2'].values[0]
            subroutes3, amount3 = create_list_of_subroutes(comm_clusters2, current_clusters_1)
            subroutes4, amount4 = create_list_of_subroutes(comm_clusters2, current_clusters_2)
            if amount4 > amount3:
                subroutes_2 = subroutes4
                amount_2 = amount4
                direction_of_result = 2
            else:
                subroutes_2 = subroutes3
                amount_2 = amount3
                direction_of_result = 1
        if amount_2 > amount_1:
            subroutes = subroutes_2
            amount = amount_2
            number_of_stops = len(comm_clusters2.split(','))
            choosen_direction = 2
        else:
            subroutes = subroutes_1
            amount = amount_1
            number_of_stops = len(comm_clusters1.split(','))
            choosen_direction = 1
            
        if len(subroutes) == 0:
            similarity_arr.append([(route_name, route_type), 0, []])
        else:                       
            similarity_arr.append([(route_name, route_type), amount, subroutes, number_of_stops, choosen_direction, direction_of_result])
    most_similar = sorted(similarity_arr, key=lambda x: x[1], reverse=True)[:number_of_similar]
    return most_similar

# Функция, которая использует в себе указанные выше и выдает отчет о наиболее похожих маршрутах
# Данная функция может использоваться в двух вариантах: 
# 1 - выдавать информацию о маршрутах, схожесть с которыми не менее threshold процентов (при использовании с флагом 'percent')
# 2 -  выдавать информацию о threshold наиболее похожих маршрутах (при использовании с флагом 'number')
def find_most_similar_routes(comm_route, type_of_result, threshold):
    if type_of_result == "percent":
        number_of_similar = 5
    else:
        number_of_similar = threshold
    similar = most_similar(comm_route, number_of_similar)
    print('Для коммерческого маршрута № '+ str(comm_route) + ' найдены следующие наиболее похожие:')
    print()
    for route in similar:
        route_pair = route[0]
        route_name = route_pair[0]
        route_type = route_pair[1]
        clusters = route[2]
        number_of_stops = route[3]
        choosen_direction = route[4]
        direction_of_result = route[5]
        similarity_percent = round(route[1] * 100 / (number_of_stops - 1))
        if type_of_result == "percent":
            if similarity_percent < threshold:
                break
        direction = ""
        if choosen_direction == 1:
            ch_direction = "Прямое направление "
        else:
            ch_direction = "Обратное направление "
        if direction_of_result == 1:
            res_direction = "прямым направлением "
        else:
            res_direction = "обратным направлением "
        type_string = ""
        if route_type == 1:
            type_string = "автобусного "
        elif route_type == 2:
            type_string = "трамвайного "
        else:
            type_string = "троллебусного "
        print(ch_direction + "маршрута №" + str(comm_route) + " совпадает с " + res_direction + type_string + 
              "маршрута № " + route_name + ", процент совпадения: ", similarity_percent, "%")
        print("Список совпадающих перегонов в виде id кластеров: " + str(clusters))
        print()

## Пример работы алгоритма

Ниже приведены примеры использования получившегося алгоритма поиска в двух описанных выше вариантах.

In [ ]:
# Данная функция может использоваться в двух вариантах: 
# 1 - выдавать информацию о маршрутах, схожесть с которыми не менее threshold процентов (при использовании с флагом 'percent')
# 2 -  выдавать информацию о threshold наиболее похожих маршрутах (при использовании с флагом 'number')

# Пример использования в варианте 2
find_most_similar_routes(68, 'number', 5)

In [ ]:
# Данная функция может использоваться в двух вариантах: 
# 1 - выдавать информацию о маршрутах, схожесть с которыми не менее threshold процентов (при использовании с флагом 'percent')
# 2 -  выдавать информацию о threshold наиболее похожих маршрутах (при использовании с флагом 'number')

# Пример использования в варианте 1
find_most_similar_routes(67, 'percent', 50)